<a href="https://colab.research.google.com/github/hajonghyun/inflearn_ML_from_the_foundation/blob/main/7_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---


# 선형회귀 이론 및 실습 # 7 Ensemble


---
---



## velog 주소

https://velog.io/@changhtun1/Python-Decision-Tree-%EC%9D%B4%EB%A1%A0-%EB%B0%8F-%EC%8B%A4%EC%8A%B5

## 유튜브 주소

https://youtu.be/OAg7vOFjVck?si=rPLVlwN58ZtPR1J3

## 연습 데이터 url

https://drive.google.com/drive/folders/149jcCyJFKKG5MFaPNWnYYqM2EkzgRz2P?usp=sharing


---
---


---
---
- 앙상블 모형의 좋은 성능을 내기 위해서는 다양한 종류의 오차를 만들어야 하고, 그러기 위해서는 다양한 알고리즘을 사용해야 한다

# 🏰 다양한 오차를 만들기 위한 방법
 1. 모델을 다양하게 ensemble
 2. 훈련 데이터를 다양하게 사용
    - 훈련 세트의 서브셋을 무작위로 구성하여 모델을 학습시키기
        - 배깅: 훈련 세트의 중복을 허용하여 샘플링을 하는 방식(통계학에선 bootstrapping이라고 불림)
        - 페이스팅: 훈련 세트의 중복을 허용하지 않고 샘플링 하는 방식
        - 전반적으로 배깅이 더 나은 성능. 교차검증이 젤 좋음.
 3. 훈련 데이터의 feature를 다양하게 사용
 ---
 ---

## ✅1. 모델을 다양하게 ensemble

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터셋 로드
iris = load_iris()
x = iris.data[:,2:] # 꽃잎의 길이, 너비만
y = iris.target

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=True,random_state=2021) # shuffle을 하지 않으면 y 00011111 앞부분만 잘림.

# 약한 학습기 구축
log_model = LogisticRegression()
rand_model = RandomForestClassifier()
svm_model = SVC()

# 앙상블 모델 구축
voting_model = VotingClassifier(
    estimators = [('lr',log_model),
                  ('rf',rand_model),
                  ('svc',svm_model),],
    voting = 'hard' #직접투표방법
)

# 모델 비교
for model in (log_model,rand_model,svm_model,voting_model):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print(model.__class__.__name__+"정확도: "+str(accuracy_score(y_test,y_pred)))


LogisticRegression정확도: 1.0
RandomForestClassifier정확도: 0.9555555555555556
SVC정확도: 1.0
VotingClassifier정확도: 1.0


# ✅2. 훈련 데이터를 다양하게 사용

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 모델 구축
bag_model = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500, # 약한 학습기(결정 트리) 500개 생성
    max_samples= 0.05, # 0.0~1.0 사이 실수 선택(실수*샘플 수) 혹은 샘플 수 지정
    bootstrap=True, # True:배깅 False: 페이스팅
    n_jobs=-1 # 사용할 CPU 코어 수
)

# 모델 학습
bag_model.fit(x_train,y_train)

# 모델 예측
y_pred = bag_model.predict(x_test)

# 모델 평가
print(bag_model.__class__.__name__,accuracy_score(y_test,y_pred))

BaggingClassifier 0.9555555555555556


## ✅3. 훈련 데이터의 feature를 다양하게 사용
- 위의 2번에서는 데이터셋의 row를 변형했다면, 3번에서는 데이터셋의 column을 변형하여 예측기에 대한 오차 다양성을 제공.

- 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링
- 랜덤 서브스페이스 방식: 훈련 샘플은 모두 사용하고, 특성만 샘플링

# 🚩랜덤포레스트

- 랜덤포레스트는 일반적으로 배깅방법을 사용한 결정트리 앙상블 모델이다.
- 그래서 BaggingClassifier에 DecisionTreeClassifier를 넣는 대신, RandomForestClassifier를 사용할 수 있다.
- 그래서 RandomForestClassifier는 DecisionTreeClassifier와 BaggingClassifier 매개변수 모두 가지고 있다.
- 랜덤포레스트 모델은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 것이 아니라, 무작위로 선택한 특성들 중에서 최선의 특성을 찾는 방식을 채택하여 무작위성을 더 가지게 된다.
- 이를 통해 약간의 편향은 손해보지만,(= 편향이 증가 = 그래프 단순 = 과소적합 위험) 더욱 다양한 트리를 만들므로 분산을 전체적으로 낮추어서(= 민감도 낮춤 = 과적합 위험 줄임) 더 훌륭한 모델을 만들 수 있다.
(분산: 데이터가 평균 주위에 흩어져 있는 정도, 분산이 크면 과적합 위험이 있다.)



In [5]:
from sklearn.ensemble import RandomForestClassifier

rnd_model = RandomForestClassifier(
    n_estimators = 500, # 예측가 500개
    max_leaf_nodes = 16, # 자식 노드의 최대 개수
    n_jobs=-1 # cpu 코어 사용 개수
)

rnd_model.fit(x_train,y_train)

y_pred = rnd_model.predict(x_test)

print("RandomForest 모델: ", accuracy_score(y_pred,y_test))

RandomForest 모델:  0.9333333333333333


## 특성 중요도 => RF 블랙박스로 알아내기 가능.

- 랜덤포레스트는 성능이 좋다는 장점말고, 특성의 상대적 중요도를 측정하기 쉽다.(트리기반 모델은 특성 중요도 제공)

- 사이킷런에서는 어떤 특성을 사용한 노드가 평균적으로 불순도를 감소시키는지 확인하여 특성 중요도를 측정하고, 훈련이 끝나고 난 뒤에 특성마다 자동으로 점수를 계산하고 저장한다.
- 저장된 값은 **featureimportances 변수**에 저장되어 있다.

### **주의해야할 점**
내가 만든 모델에 대해서만 적용되는 특성 중요도인 것이지, 전체적인 것에 모두 적용되는 것이 아니다!!!

In [10]:
# 특성의 상대적 중요도 => featuresimportances 변수
iris = load_iris()
x = iris.data[:,:]
y = iris.target

rnd_model = RandomForestClassifier(
    n_estimators = 500,
    n_jobs=-1
)
rnd_model.fit(x,y)

for feature_name,feature_impo in zip(iris['feature_names'],rnd_model.feature_importances_):
    print(feature_name,' : ',feature_impo)

sepal length (cm)  :  0.09777817211823935
sepal width (cm)  :  0.022718811029106997
petal length (cm)  :  0.43857573562709223
petal width (cm)  :  0.4409272812255614


# 🚩엑스트라 트리
랜덤포레스트보다 편향이 더 심하고, 분산은 더욱 낮춘 모델. 과적합 위험 감소.
- RandomForestClassifier와 ExtraTreesClassifier 중 어떤 것이 더 좋을지는 판단하기 어렵기 때문에, 교차검증을 통해서 서로 비교해보고, 더 나은 모델을 선택하여 그리드 탐색방법을 사용해 하이퍼파라미터 튜닝을 한다.